# Agenda, week 4

1. Recap and Q&A
    - Oil prices!
2. Text strings
    - The `str` accessor
    - Cleaning dirty integer data
    - Textual statistics
    - Trimming strings
3. Dates and times
    - Date and time dtypes
    - Parsing CSV files with times
    - Time deltas
    - Time series
    - Resampling

In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [4]:
filename = 'oil-prices-master/data/wti-daily.csv'

df = pd.read_csv(filename)
df.head()

,Date,Price
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87


In [5]:
# What was the highest-ever price of oil (as per WTI)?

df['Price'].max()   # this is the highest price ever

145.31

In [6]:

df['Price'] == df['Price'].max()  # this returns a boolean series

0       False
1       False
2       False
3       False
4       False
        ...  
9222    False
9223    False
9224    False
9225    False
9226    False
Name: Price, Length: 9227, dtype: bool

In [8]:
# Use .loc with a row selector and a column selector
# our row selector will be our boolean series


df.loc[
    df['Price'] == df['Price'].max(),   # row selector
    'Date' # column selector
]

5678    2008-07-03
Name: Date, dtype: object

In [10]:
# what was the minimum price ever found of WTI?


df.loc[
    df['Price'] == df['Price'].min(),   # row selector
    ['Date', 'Price']                              # column selector
]

,Date,Price
8643,2020-04-20,-36.98


In [11]:
# what were the 10 most recent values for oil prices?

df.tail(5)

,Date,Price
9222,2022-08-09,93.18
9223,2022-08-10,94.68
9224,2022-08-11,97.02
9225,2022-08-12,94.86
9226,2022-08-15,92.24


In [12]:
df.tail(20)

,Date,Price
9207,2022-07-19,106.12
9208,2022-07-20,104.45
9209,2022-07-21,98.44
9210,2022-07-22,97.71
9211,2022-07-25,99.83
9212,2022-07-26,97.74
9213,2022-07-27,100.03
9214,2022-07-28,99.11
9215,2022-07-29,101.31
9216,2022-08-01,96.59
